In [1]:
import pandas as pd
import numpy as np

import re
import openpyxl

from IPython.core.debugger import set_trace

from datetime import date
from dateutil.relativedelta import relativedelta
from collections import defaultdict

## Setup dataframes

In [2]:
# General config

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

display_cols = [
    'nappi_code', 'name', 'unit', 'size', 'quantity', 'sep', 'unit_price', 'effective_date'
]


In [3]:
# Create dataframe

data_prefix = 'data/'
data_file = 'mpr-cleaned-filled-down.csv'

header_names = [
    'license_no', 'applicant_name', 'reg_no', 'nappi_code', 'atc_4', 'schedule',
    'name', 'active_ingredient', 'strength', 'unit', 'form', 'size', 'quantity', 'man_price',
    'logistics_fee', 'vat', 'sep', 'unit_price', 'effective_date', 'status', 'u',
    'original_generic','sales_volume'
]

df = pd.read_csv(
    data_prefix + data_file,
    sep=',',
    header=0,
    names=header_names,
    parse_dates=['effective_date'])

# Make a copy of df to use when exporting
original_df = df

# Convert some columns to numbers, creating NaNs on errors
number_cols = [
    'size', 'quantity', 'sep', 'unit_price',
    'man_price', 'logistics_fee', 'vat'
]

df[number_cols] = df[number_cols].apply(pd.to_numeric, errors='coerce')

# TODO: 
# Check number of NaNs before and after converting

In [4]:
# Before we drop any rows, we create a cell with all the active ingredients
# active-strength-unit

# We need an ingredient column wiith strengths and units
df['full_ingredient'] = df['active_ingredient'] + " " + df['strength'] + " " + df['unit']
df.head()

ingredient_df = (df.groupby(
    ['nappi_code', 'effective_date', 'full_ingredient'])
    .size()
    .to_frame('Count')
    .reset_index()
    .sort_values('nappi_code')
)

groupby_nappi = ingredient_df.groupby(['nappi_code', 'effective_date'])

# .transform casts back to shape of original df the group it's operating on was created from
# We combine them with ;; as a seperator
combine_ingredients = lambda x: ';;'.join(x.values.tolist())
ingredient_df['combined_ingredients'] = groupby_nappi['full_ingredient'].transform(combine_ingredients)

In [5]:
# Drop the data we can't use

# Rows with NaN in certain cols
drop_nan_cols = ['sep', 'effective_date', 'nappi_code']
df = df.dropna(axis=0, subset=drop_nan_cols)


# 24 rows with SEP == 0.00, all in 2010. Assuming erroneous
idx = df['sep'] != 0
df = df.loc[idx, :]

# Filter out rows with invvalid NAPPI codes
idx = df['nappi_code'].str.len() == 9
df = df.loc[idx, :]

print "Rows in df: %s" % (len(df.index))

Rows in df: 71768


In [6]:
# Create a year colum. This could be done in a better way.

get_year = lambda x: pd.Series(x[0:4])
df['year'] = df['effective_date'].apply(get_year).apply(pd.to_numeric, errors='coerce')

In [9]:
# Read Medikredit file

medikredit_pd_file = 'medikredit20170830.txt'

medikredit_nappi_to_size = defaultdict()
medikredit_dict = defaultdict(list)

with open(data_prefix + medikredit_pd_file) as f:
    next(f)
    for line in f:
        nappi_code = line[10:19]
        size_str = line[79:88]
        try:
            int(nappi_code)
        except:
            continue
        try:
            size = np.float64(size_str[:-2] + "." + size_str[-2:])
        except:
            size = None
        medikredit_nappi_to_size[nappi_code] = size
        
        medikredit_dict['nappi_code'].append(nappi_code)
        medikredit_dict['size'].append(size)
        
f.close()

medikredit_df = pd.DataFrame.from_dict(medikredit_dict)

In [10]:
# Which items don't occur in the Medikredit data
mpr_only_df = df[
    (~df['nappi_code'].isin(medikredit_df['nappi_code']))
]

len(mpr_only_df.groupby(['nappi_code']).size().to_frame('Count').reset_index())

1874

## Reduce to reliable data

In [11]:
# Drop the years we're not interested in
df.groupby('year').size()

df = df[(df['year'] >= 2008) & (df['year'] <= 2017)]

In [12]:
# No. of unique NAPPI codes
len(df.groupby('nappi_code').size())

9271

In [13]:
# Frequency of NAPPI codes over the years.
# Only use years with many items to determine the frequesncy of occurrrence
# 2011 has 1/10th of the records of other years. Exclude in calculating frequency
# Check codes in each year: df.groupby('year').size()

years = [2009, 2010, 2012, 2013, 2014, 2015, 2016]

# Some years occur more than once. Let's get unique instances by grouping by years too for now.
# We could also adjust the get_perc function the provide for this

nappi_year_df = (
    df.groupby(['nappi_code', 'year'])
    .size()
    .to_frame('Count')
    .reset_index()
)

groupby_nappi = nappi_year_df.groupby('nappi_code')

# Transform casts back to shape of original df the group it's operating on was created from
calc_freq = lambda x: float(np.sum(x.isin(years))) / len(years)
nappi_year_df['freq'] = groupby_nappi['year'].transform(calc_freq)

In [14]:
# We only want to merge the perc_occurrence. Group by and drop the other columns
freq_df = (
    nappi_year_df.groupby(['nappi_code', 'freq'])
    .size()
    .to_frame('Count')
    .reset_index()
)

del freq_df['Count']

# Delete the freq column if it already exists
try:
    del df['freq']
except KeyError:
    pass

df = pd.merge(
    df, freq_df,
    how='left', on='nappi_code',
    sort=True,
    copy=True,
    indicator=False)

In [15]:
# NAPPI codes for which the size doesn't change over the years

nappi_size_df = (
    df.groupby(['nappi_code', 'size'])
    .size()
    .to_frame('Count')
    .reset_index()
    .sort_values('nappi_code')
)

size_change_df = nappi_size_df[nappi_size_df.duplicated('nappi_code', keep=False)]

len(df[(~df['nappi_code'].isin(size_change_df['nappi_code']))].groupby(['nappi_code']).size())

7584

In [16]:
# NAPPI codes for which ingredients don't change over years
nappi_ingredient_df = (
    ingredient_df.groupby(['nappi_code', 'combined_ingredients'])
    .size()
    .to_frame('Count')
    .reset_index()
    .sort_values('nappi_code')
)

ingredient_change_df = nappi_ingredient_df[nappi_ingredient_df.duplicated('nappi_code', keep=False)]

len(df[(~df['nappi_code'].isin(ingredient_change_df['nappi_code']))].groupby(['nappi_code']).size())

6144

In [17]:
# Number of NAPPI codes in the final dataset
# freq > 0.5 translates to at least 4 of the 7 years.

len(
    df[
        (~df['nappi_code'].isin(size_change_df['nappi_code'])) &
        (~df['nappi_code'].isin(ingredient_change_df['nappi_code'])) &
        (df['freq'] > 0.50) &
        (df['nappi_code'].isin(medikredit_df['nappi_code']))
    ]
    .groupby(['nappi_code'])
    .size()
)

3201

In [21]:
# File to use for analysis

simplified_df = df[
    (~df['nappi_code'].isin(size_change_df['nappi_code'])) &
    (~df['nappi_code'].isin(ingredient_change_df['nappi_code'])) &
    (df['freq'] > 0.50) &
    (df['nappi_code'].isin(medikredit_df['nappi_code']))
]

simplified_df['license_no'].apply(str)

simplified_df.to_csv(data_prefix + 'mpr-reliable-data.csv', encoding='utf-8', index=False)

In [22]:
# There are some issues during import with the date in this file
# Needs to be fixed
# Export csv above and save as xlsx before importing

# File to use in import of data


# export_df = original_df[original_df['nappi_code'].isin(simplified_df['nappi_code'])]

# export_df['license_no'].apply(str)

# writer = pd.ExcelWriter(data_prefix + 'mpr-data-simplified-for-import.xlsx')
# export_df.to_excel(writer, 'Sheet1', encoding='utf-8', index=False)
# writer.save()